In [ ]:
from pathlib import Path
import sys

DATA_DIR = Path.home() / "projects" / "data"

csv_file = DATA_DIR / "raw" /"GSE290585_SeSaMeBeta_MM285_BS.csv" # path to mouse csv file

print(sys.executable)
print(csv_file.exists())

OUTPUT_DIR = DATA_DIR / "processed"

/home/jullid/projects/mouse-DNA-project/.venv/bin/python
True


In [2]:
import pandas as pd

df = pd.read_csv(csv_file)

with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df.head(20))


,probe_ID,Liver_Female_12_weeks_BS,Blood_Male_24_weeks_BS,Blood_Male_36_weeks_BS,Liver_Female_52_weeks_BS,Blood_Male_8_weeks_BS_ACEseq,Testis_Male_12_weeks_BS,Skin_Female_24_weeks_BS,Liver_Male_36_weeks_BS,Skin_Female_52_weeks_BS,Ears_Male_8_weeks_BS,Spleen_Male_20_weeks_BS,Colon_Male_8_weeks_BS,Eye_Male_8_weeks_BS,Adrenal_Female_8_weeks_BS,Blood_Female_12_weeks_BS,Kidney_Male_12_weeks_BS,Tail_Male_12_weeks_BS,Heart_Male_8_weeks_BS_ACEseq,Skin_Male_8_weeks_BS,Testis_Male_36_weeks_BS,Liver_Male_76_weeks_BS,Tail_Male_12_weeks_BS.1,Brain_Cortex_Male_20_weeks_BS,Eye_Female_8_weeks_BS,Skin_Female_12_weeks_BS,Liver_Female_36_weeks_BS,Blood_Female_76_weeks_BS,Spleen_Male_16_weeks_BS,Liver_Male_8_weeks_BS,Lung_Male_8_weeks_BS,Spleen_Male_8_weeks_BS,Tail_Male_8_weeks_BS,Mammary_Glands_Male_8_weeks_BS,Blood_Female_8_weeks_BS,Colon_Female_8_weeks_BS,Brain_Cortex_Female_8_weeks_BS,Urinary_Bladder_Female_8_weeks_BS,Spinal_Cord_Female_8_weeks_BS,Skin_Female_8_weeks_BS,Pancreas_Female_8_weeks_BS,Liver_Male_24_weeks_BS,Testis_Male_24_weeks_BS,Skin_Male_24_weeks_BS,Blood_Female_24_weeks_BS,Kidney_Male_16_weeks_BS,Tail_Male_16_weeks_BS,Blood_Male_52_weeks_BS,Testis_Male_52_weeks_BS,Skin_Male_52_weeks_BS,Kidney_Male_16_weeks_BS.1,Brain_Cortex_Female_27_weeks_BS,Spleen_Female_27_weeks_BS,Blood_Female_52_weeks_BS,Brain_Cortex_Male_12_weeks_BS,Diaphragm_Male_8_weeks_BS,Thymus_Male_8_weeks_BS,Skin_Female_76_weeks_BS,Lung_Female_8_weeks_BS,Diaphragm_Female_8_weeks_BS,Brain_Cortex_Male_8_weeks_BS,Urinary_Bladder_Male_8_weeks_BS,Ears_Female_8_weeks_BS,Skin_Male_36_weeks_BS,Skin_Male_76_weeks_BS,Brain_Cortex_Male_16_weeks_BS,Kidney_Male_8_weeks_BS_ACEseq,Pancreas_Male_8_weeks_BS,Kidney_Male_20_weeks_BS,Skin_Female_36_weeks_BS,Liver_Female_76_weeks_BS,Tail_Male_20_weeks_BS,Blood_Male_76_weeks_BS,Bile_Duct_Male_8_weeks_BS,Liver_Female_8_weeks_BS,Testis_Male_8_weeks_BS,Brain_Cortex_Male_16_weeks_BS.1,Tail_Female_27_weeks_BS,Spleen_Male_12_weeks_BS,Liver_Male_12_weeks_BS,Heart_Female_8_weeks_BS,Mammary_Glands_Female_8_weeks_BS,Blood_Male_12_weeks_BS,Tail_Male_16_weeks_BS.1,Brain_Cortex_Male_12_weeks_BS.1,Adrenal_Male_8_weeks_BS,Liver_Female_24_weeks_BS,Colon_Male_12_weeks_BS,Kidney_Female_27_weeks_BS,Liver_Male_52_weeks_BS,Colon_Male_16_weeks_BS,Bile_Duct_Female_8_weeks_BS,Femur_Female_8_weeks_BS,Retina_Female_8_weeks_BS,Thymus_Female_8_weeks_BS,Sciatic_Nerve_Female_8_weeks_BS,Optic_Nerve_Female_8_weeks_BS,Blood_Female_36_weeks_BS,Liver_Male_16_weeks_BS,Colon_Female_27_weeks_BS,Liver_Female_27_weeks_BS,Colon_Male_12_weeks_BS.1,Kidney_Male_12_weeks_BS.1,Femur_Male_8_weeks_BS,Liver_Male_12_weeks_BS.1,Liver_Male_12_weeks_BS.2,Colon_Male_20_weeks_BS,Liver_Male_20_weeks_BS,Kidney_Female_8_weeks_BS,Optic_Nerve_Male_8_weeks_BS,Tail_Female_8_weeks_BS,Liver_Male_16_weeks_BS.1,Spleen_Male_16_weeks_BS.1,Sciatic_Nerve_Male_8_weeks_BS,Testis_Male_76_weeks_BS,Cerebellum_Male_8_weeks_BS,Subcortical_Brain_Male_8_weeks_BS,Cerebellum_Male_8_weeks_BS_ACEseq,Subcortical_Brain_Male_8_weeks_BS.1,Subcortical_Brain_Female_8_weeks_BS,Subcortical_Brain_Female_8_weeks_BS.1,Brain_Cortex_Male_8_weeks_BS.1,Optic_Nerve_Male_8_weeks_BS.1,Lung_Male_8_weeks_BS.1,Brain_Cortex_Male_8_weeks_BS.2,Optic_Nerve_Male_8_weeks_BS.2,Lung_Female_76_weeks_BS,Eye_Female_76_weeks_BS,Brain_Cortex_Female_8_weeks_BS.1,Optic_Nerve_Female_8_weeks_BS.1,Cerebellum_Female_8_weeks_BS,Brain_Cortex_Female_8_weeks_BS.2,Optic_Nerve_Female_8_weeks_BS.2,Cerebellum_Female_8_weeks_BS.1,Optic_Nerve_Male_76_weeks_BS,Cerebellum_Male_76_weeks_BS,Brain_Cortex_Male_76_weeks_BS,Optic_Nerve_Male_76_weeks_BS.1,Spleen_Female_8_weeks_BS,Cerebellum_Male_76_weeks_BS.1,Brain_Cortex_Female_76_weeks_BS,Optic_Nerve_Female_76_weeks_BS,Eye_Male_8_weeks_BS.1,Cerebellum_Female_76_weeks_BS,Brain_Cortex_Female_76_weeks_BS.1,Eye_Female_8_weeks_BS.1,Subcortical_Brain_Male_76_weeks_BS,Subcortical_Brain_Male_76_weeks_BS.1,Subcortical_Brain_Female_76_weeks_BS,Heart_Male_8_weeks_BS,Heart_Male_8_weeks_BS.1,Heart_Female_8_weeks_BS.1,Heart_Female_8_week

In [3]:
df.columns

Index(['probe_ID', 'Liver_Female_12_weeks_BS', 'Blood_Male_24_weeks_BS',
       'Blood_Male_36_weeks_BS', 'Liver_Female_52_weeks_BS',
       'Blood_Male_8_weeks_BS_ACEseq', 'Testis_Male_12_weeks_BS',
       'Skin_Female_24_weeks_BS', 'Liver_Male_36_weeks_BS',
       'Skin_Female_52_weeks_BS',
       ...
       'Colon_Female_8_weeks_BS.1', 'Kidney_Female_8_weeks_BS.1',
       'Spleen_Female_8_weeks_BS.1', 'Ears_Female_8_weeks_BS.1',
       'Tail_Female_8_weeks_BS.1', 'Colon_Female_8_weeks_BS.2',
       'Kidney_Female_8_weeks_BS.2', 'Spleen_Female_8_weeks_BS.2',
       'Ears_Female_8_weeks_BS.2', 'Tail_Female_8_weeks_BS.2'],
      dtype='object', length=268)

In [4]:
#Regex version

import re
import warnings

def simplify_colname_regex_version(name: str, n_underscores: int = 4) -> str:
    """
    Keep only the part before the nth underscore from the end.
    Preserve trailing .<number> suffix (e.g., .1, .2).
    Warn and return original if fewer than n_underscores underscores.
    """
    # 1) Preserve trailing ".<digits>" if present
    m = re.search(r"(\.\d+)$", name)
    suffix = m.group(1) if m else ""
    core = name[:-len(suffix)] if suffix else name

    # 2) Count underscores in the core name
    underscore_count = core.count("_")
    if underscore_count < n_underscores:
        warnings.warn(
            f"Column '{name}' has only {underscore_count} underscores (< {n_underscores}); leaving unchanged."
        )
        return name

    # 3) Split from the right and keep the left part
    simplified = core.rsplit("_", n_underscores)[0]

    # 4) Add suffix back
    return simplified + suffix


In [5]:
# non regex (a lot more readable for mere mortals)

def simplify_colname(name: str, n_underscores: int = 4) -> str:
    # 1) Split on dots
    parts = name.split(".")

    # 2) Check for trailing numeric suffix
    if len(parts) > 1 and parts[-1].isdigit():
        suffix = "." + parts[-1]
        core = ".".join(parts[:-1])
    else:
        suffix = ""
        core = name


    # SPECIAL CASE: ACE-seq columns
    if core.endswith("_ACEseq"):
        core = core.replace("_ACEseq", "")

    # 3) Count underscores
    underscore_count = core.count("_")
    if underscore_count < n_underscores:
        warnings.warn(
            f"Column '{name}' has only {underscore_count} underscores (< {n_underscores}); leaving unchanged."
        )
        return name

    # 4) Split from the right and keep left part
    simplified = core.rsplit("_", n_underscores)[0]

    # 5) Add suffix back
    return simplified + suffix


In [6]:
# Test if working so far

tests = [
    "Blood_Male_36_weeks_BS",
    "Tail_Female_8_weeks_BS.2",
    "probe_ID"
]

for t in tests:
    print(t, "->", simplify_colname(t))


Blood_Male_36_weeks_BS -> Blood
Tail_Female_8_weeks_BS.2 -> Tail.2
probe_ID -> probe_ID


/tmp/ipykernel_3271020/3965563143.py:23: UserWarning: Column 'probe_ID' has only 1 underscores (< 4); leaving unchanged.
  warnings.warn(


In [7]:
# visually inspect the columns
original_columns = df.columns
original_columns[:10]

# we do not want to change the first column, so here we separate them
first_col = df.columns[0]
other_cols = df.columns[1:]

# loop through the columns
simplified_other_cols = []

for col in other_cols:
    new_col = simplify_colname(col, n_underscores=4)
    simplified_other_cols.append(new_col)


# Reconstruct the full column list with first column:
new_columns = [first_col] + simplified_other_cols


In [8]:
# critical inspection step to see if it works
for old, new in zip(df.columns[-10:], new_columns[-10:]):
    print(f"{old}  ->  {new}")    

Colon_Female_8_weeks_BS.1  ->  Colon.1
Kidney_Female_8_weeks_BS.1  ->  Kidney.1
Spleen_Female_8_weeks_BS.1  ->  Spleen.1
Ears_Female_8_weeks_BS.1  ->  Ears.1
Tail_Female_8_weeks_BS.1  ->  Tail.1
Colon_Female_8_weeks_BS.2  ->  Colon.2
Kidney_Female_8_weeks_BS.2  ->  Kidney.2
Spleen_Female_8_weeks_BS.2  ->  Spleen.2
Ears_Female_8_weeks_BS.2  ->  Ears.2
Tail_Female_8_weeks_BS.2  ->  Tail.2


In [9]:
# Assign new column names to the df
df.columns = new_columns

# visual inspection
df.columns[:10]
df.columns.is_unique # will return false due to current design choices



False

In [10]:
with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df.head(20))


,probe_ID,Liver,Blood,Blood,Liver,Blood,Testis,Skin,Liver,Skin,Ears,Spleen,Colon,Eye,Adrenal,Blood,Kidney,Tail,Heart,Skin,Testis,Liver,Tail.1,Brain_Cortex,Eye,Skin,Liver,Blood,Spleen,Liver,Lung,Spleen,Tail,Mammary_Glands,Blood,Colon,Brain_Cortex,Urinary_Bladder,Spinal_Cord,Skin,Pancreas,Liver,Testis,Skin,Blood,Kidney,Tail,Blood,Testis,Skin,Kidney.1,Brain_Cortex,Spleen,Blood,Brain_Cortex,Diaphragm,Thymus,Skin,Lung,Diaphragm,Brain_Cortex,Urinary_Bladder,Ears,Skin,Skin,Brain_Cortex,Kidney,Pancreas,Kidney,Skin,Liver,Tail,Blood,Bile_Duct,Liver,Testis,Brain_Cortex.1,Tail,Spleen,Liver,Heart,Mammary_Glands,Blood,Tail.1,Brain_Cortex.1,Adrenal,Liver,Colon,Kidney,Liver,Colon,Bile_Duct,Femur,Retina,Thymus,Sciatic_Nerve,Optic_Nerve,Blood,Liver,Colon,Liver,Colon.1,Kidney.1,Femur,Liver.1,Liver.2,Colon,Liver,Kidney,Optic_Nerve,Tail,Liver.1,Spleen.1,Sciatic_Nerve,Testis,Cerebellum,Subcortical_Brain,Cerebellum,Subcortical_Brain.1,Subcortical_Brain,Subcortical_Brain.1,Brain_Cortex.1,Optic_Nerve.1,Lung.1,Brain_Cortex.2,Optic_Nerve.2,Lung,Eye,Brain_Cortex.1,Optic_Nerve.1,Cerebellum,Brain_Cortex.2,Optic_Nerve.2,Cerebellum.1,Optic_Nerve,Cerebellum,Brain_Cortex,Optic_Nerve.1,Spleen,Cerebellum.1,Brain_Cortex,Optic_Nerve,Eye.1,Cerebellum,Brain_Cortex.1,Eye.1,Subcortical_Brain,Subcortical_Brain.1,Subcortical_Brain,Heart,Heart.1,Heart.1,Heart.2,Heart,Heart,Lung.2,Lung.1,Femur.1,Thymus.1,Femur.2,Heart.1,Lung.2,Lung,Eye.2,Thymus.2,Femur.1,Thymus.1,Femur,Testis.1,Lung.1,Lung.1,Testis.1,Liver.1,Testis.2,Liver.2,Liver.1,Eye.2,Thymus.2,Liver.2,Heart.1,Testis.2,Eye,Liver.1,Brain_Cortex.1,Femur.1,Eye.1,Thymus,Liver.2,Femur,Eye.1,Ears,Liver.1,Femur.1,Ears.1,Liver.2,Spinal_Cord,Cerebellum.1,Subcortical_Brain.1,Femur.2,Stomach,Uterus,Pancreas.1,Stomach,Uterus.1,Cerebellum.2,Subcortical_Brain.2,Stomach.1,Diaphragm.1,Pancreas.2,Uterus.2,Urinary_Bladder.1,Mammary_Glands.1,Retina.1,Stomach.2,Diaphragm.2,Pancreas.3,Uterus.3,Urinary_Bladder.2,Mammary_Glands.2,Retina.2,Diaphragm.1,Pancreas.1,Urinary_Bladder.1,Mammary_Glands.1,Retina,Retina.1,Bile_Duct.1,Stomach.1,Diaphragm.2,Pancreas.2,Urinary_Bladder.2,Mammary_Glands.2,Retina.2,Retina.3,Bile_Duct.1,Bile_Duct.2,Bile_Duct.2,Spinal_Cord.1,Spinal_Cord.2,Spinal_Cord.1,Spinal_Cord.2,Sciatic_Nerve.1,Sciatic_Nerve.2,Sciatic_Nerve.1,Adrenal.1,Adrenal.2,Adrenal.1,Adrenal.2,Colon.1,Kidney,Spleen.1,Ears.1,Tail.1,Colon.2,Kidney.1,Spleen.2,Ears.2,Tail.2,Colon.1,Kidney.1,Spleen.1,Ears.1,Tail.1,Colon.2,Kidney.2,Spleen.2,Ears.2,Tail.2
0,cg00101675_BC21,0.705,0.783,0.793,0.720,0.827,0.733,0.739,0.735,0.696,0.705,0.815,0.725,0.813,0.694,0.808,0.650,0.593,0.597,0.643,0.710,0.593,0.629,0.736,0.766,0.629,0.638,0.784,0.813,0.709,0.725,0.808,0.693,0.837,0.861,0.750,0.758,0.590,0.666,0.699,0.682,0.714,0.751,0.691,0.792,0.703,0.647,0.802,0.684,0.663,0.654,0.742,0.790,0.815,0.783,0.707,0.801,0.639,0.657,0.690,0.718,0.577,0.637,0.742,0.795,0.814,0.724,0.781,0.754,0.713,0.757,0.750,0.860,0.631,0.677,0.690,0.751,0.716,0.759,0.678,0.621,0.758,0.834,0.644,0.753,0.782,0.754,0.742,0.774,0.665,0.785,0.600,0.510,0.773,0.831,0.723,0.775,0.798,0.666,0.659,0.729,0.743,0.717,0.535,0.680,0.718,0.797,0.756,0.694,0.782,0.746,0.758,0.800,0.749,0.713,0.837,0.816,0.830,0.766,0.746,0.742,0.847,0.802,0.775,0.832,0.799,0.771,0.877,0.847,0.823,0.890,0.822,0.831,0.899,0.839,0.858,0.868,0.787,0.866,0.870,0.833,0.807,0.841,0.887,0.851,0.854,0.817,0.830,0.845,0.736,0.741,0.740,0.700,0.705,0.684,0.736,0.726,0.635,0.860,0.694,0.750,0.733,0.842,0.897,0.864,0.740,0.921,0.696,0.831,0.731,0.732,0.733,0.722,0.710,0.708,0.729,0.827,0.846,0.758,0.727,0.785,0.841,0.767,0.756,0.588,0.803,0.792,0.752,0.636,0.814,0.733,0.688,0.568,0.742,0.725,0.718,0.888,0.775,0.675,0.753,0.395,0.770,0.680,0.359,0.778,0.626,0.521,0.660,0.654,0.309,0.648,0.727,0.747,0.575,0.701,0.722,0.405,0.644,0.696,0.764,0.658,0.619,0.561,0.692,0.744,0.711,0.600,0.551,0.691,0.681,0.581,0.724,0.771,0.732,0.602,0.581,0.573,0.620,0.650,0.570,0.633,0.557,0.657,0.527,0.593,0.525,0.609,0.626,0.680,0.598,0.687,0.530,0.501,0.

In [12]:
output_path = OUTPUT_DIR / (csv_file.stem + "_simplified_columns.csv")

df.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")

Saved to: /home/jullid/projects/data/processed/GSE290585_SeSaMeBeta_MM285_BS_simplified_columns.csv
